In [1]:
import chromadb

client = chromadb.PersistentClient(path="pocs/chroma_db")
print(client.list_collections())

[Collection(name=patient1), Collection(name=langchain)]


In [2]:
import uvicorn
import websockets
import asyncio
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import dotenv
from langchain_cohere import ChatCohere
import os
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
import json
from langchain_cohere import CohereEmbeddings

cohere_api_key = os.getenv('COHERE_API_KEY')

app = FastAPI()
session_memories = {}

embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-english-v3.0",
)

retriever = Chroma(persist_directory="pocs/chroma_db",collection_name='patient1', embedding_function=embeddings).as_retriever()

C:\Users\MSI\AppData\Local\Temp\ipykernel_13992\4032817136.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  retriever = Chroma(persist_directory="pocs/chroma_db",collection_name='patient1', embedding_function=embeddings).as_retriever()


In [3]:
retriever.invoke("haemoglobin pathology report")

[Document(metadata={'source': '../uploads/patient1/patient1_analysis.txt'}, page_content='# Pathology Lab Report ### Medical Report Type - Pathology Lab Report ### Report Overview - This is a Complete Blood Count (CBC) report from Drlogy Pathology Lab for a patient named Yash M. Patel. - The report includes results for Hemoglobin, RBC count, Blood Indices, WBC count, Differential WBC count, and Platelet count, along with reference values and units'),
 Document(metadata={'source': '../uploads/patient1/patient1_analysis.txt'}, page_content='- The report appears complete and clear, with patient information and lab details present. ### Key Findings - **Hemoglobin (Hb):** 12.5 g/dL (Low; Reference range: 13.0-17.0 g/dL) - Mildly low. - **Packed Cell Volume (PCV):** 57.5 % (High; Reference range: 40-50 %) - Mildly elevated'),
 Document(metadata={'source': '../uploads/patient1/patient1_analysis.txt'}, page_content='### Research Context - **Anemia Diagnosis and Treatment:** Anemia is a common 

In [4]:
chat = ChatCohere(cohere_api_key=cohere_api_key)
str_out = StrOutputParser()

prompt_template = PromptTemplate(
    input_variables=["history", "context", "question"],
    template="""
You are a really sympathetic and caring medical assistant for question-answering tasks. 
Your name is 'Eva'. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

### Conversation History:
{history}

### Retrieved Context:
{context}

### User Question:
{question}

### Eva's Response:
"""
)

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

prompt_template = PromptTemplate(
    input_variables=["history", "context", "question"],
    template="""
You are a really sympathetic and caring medical assistant for question-answering tasks. 
Your name is 'Eva'. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

### Conversation History:
{history}

### Retrieved Context:
{context}

### User Question:
{question}

### Eva's Response:
"""
)
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

chain = (
    {
        "history": RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt_template
    | chat
    | str_out
)

C:\Users\MSI\AppData\Local\Temp\ipykernel_13992\1977704862.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)


In [5]:
chain.invoke("tell me about the numbers in pathology report")

"Sure, I'd be happy to explain the numbers in the pathology report.\n\nThe report includes several blood test results with corresponding reference values and units:\n- **Hemoglobin (Hb):** 12.5 g/dL - This value is considered mildly low, as the reference range for a healthy individual is typically between 13.0-17.0 g/dL.\n- **Packed Cell Volume (PCV) or Hematocrit:** 57.5% - This is mildly elevated, with the normal reference range being 40-50%.\n- **Platelet Count:** 150000 cumm - This is at the lower limit of the normal range, which is 150000-410000 cumm.\n\nThese numbers are essential in diagnosing any potential medical conditions. In this case, the low hemoglobin level indicates anemia, which is also suggested in the report's interpretation. The mildly elevated PCV could be a cause for further investigation as well.\n\nI hope that helps clarify the report's findings, Yash. If you have any more questions or concerns about the report, feel free to ask."